In [3]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv

#scientific computing
import matplotlib
import math
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats
#ipython add-ons
from IPython.parallel import Client
from IPython.display import Image
import multiprocessing
import nibabel as nib
%matplotlib inline

In [4]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'subjects.txt'
sub_list = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)
exp_list = ['ser_4mm','sim_4mm','loc']

In [6]:
thresh = .5
exp = exp_list[1]
print exp
for sub in sub_list:
    for run in range(1,4):
        
        if sub == 'fd_127':
            if run == 1:
                run_idx = 2
            elif run == 2:
                run_idx = 3
        elif exp == 'sim_4mm' and sub == 'fd_133':
            if run == 2:
                run_idx = 3
            else:
                run_idx = run
        else:
            run_idx = run
            
        art_f  = home_dir + '/analysis/' + exp + '/'+ sub + '/preproc/run_' + str(run) + '/artifacts.csv'
        if os.path.exists(art_f):
            art = pd.read_csv(art_f)
            
            motion = home_dir + '/data/' + sub + '/func/sim/run_' + str(run_idx)  + '_mc.nii.gz_rel.rms'
            motion = np.loadtxt(motion)
            m = motion[8:(8 + art.shape[0])]
            art['motion']  = np.array(m > .5).astype(int)

            art.to_csv(art_f,index = False)
        else:
            print sub,run

sim_4mm
fd_127 3
fd_133 3


In [7]:
#now update the motion parameters file
exp = exp_list[1]
for sub in sub_list:
    for run in range(1,4):
        
        if sub == 'fd_127':
            if run == 1:
                run_idx = 2
            elif run == 2:
                run_idx = 3
        elif exp == 'sim_4mm' and sub == 'fd_133':
            if run == 2:
                run_idx = 3
            else:
                run_idx = run
        else:
            run_idx = run
        
        analysis_dir = home_dir + '/analysis/' + exp + '/'+ sub + '/preproc/run_' + str(run) + '/'
        motion_f  = analysis_dir + '/realignment_params.csv'
        
        if os.path.exists(analysis_dir):

            arch_motion_f  = analysis_dir + '/realignment_params_2nd_pass.csv'
            if not os.path.exists(arch_motion_f):
                os.rename(motion_f, arch_motion_f)

            #load df to get shape of 
            art_f  = analysis_dir + '/artifacts.csv'
            art = pd.read_csv(art_f)
            motion = home_dir + '/data/' + sub + '/func/sim/run_' + str(run_idx)  + '_mc.nii.gz.par'
            rel = home_dir + '/data/' + sub + '/func/sim/run_' + str(run_idx)  + '_mc.nii.gz_rel.rms'
            absolute = home_dir + '/data/' + sub + '/func/sim/run_' + str(run_idx)  + '_mc.nii.gz_abs.rms'

            motion = np.loadtxt(motion)
            rel = np.loadtxt(rel)
            absolute = np.loadtxt(absolute)

            labels = ['rot_x','rot_y','rot_z','trans_x','trans_y','trans_z']

            df = {}
            df['displace_abs'] = absolute[8:(8 + art.shape[0])]
            df['displace_rel'] = rel[8:(8 + art.shape[0])]
            for i in range(6):
                df[labels[i]] = motion[8:(8 + art.shape[0]),i]
            df = pd.DataFrame(df)
            df.to_csv(motion_f)
        else:
            print sub,run

fd_127 3
fd_133 3


In [37]:
def reset_art_thresh(in_tuple):
    sub,exp,run = in_tuple

    
    path = home_dir + '/analysis/' + exp + '_4mm/' + sub + '/preproc/run_' + str(run) 
    f =  path + '/unsmoothed_timeseries.nii.gz'
    mask = path + '/functional_mask.nii.gz'
    old_art = path + '/artifacts_3_thresh.csv'

    if os.path.exists(f) and not os.path.exists(old_art):
        # Load the timeseries and mask files
        ts = nib.load(f).get_data()
        mask = nib.load(mask).get_data().astype(bool)

        # Normalize the timeseries using robust statistics
        brain_ts = ts[mask]
        med_ts = np.median(brain_ts, axis=0)
        mad = np.median(np.abs(med_ts - np.median(brain_ts)))
        norm_ts = (med_ts - np.median(brain_ts)) / mad

        # Find the intensity artifacts
        art_intensity = np.abs(norm_ts) > 4

        art_f  = path + '/artifacts.csv'
        if os.path.exists(art_f):
            art = pd.read_csv(art_f)

        art.to_csv(old_art,index = False)

        art['intensity'] = np.array(art_intensity).astype(int)
        art.to_csv(art_f,index = False)

In [38]:
exps = ['ser','sim']
in_tuples = []
for sub in sub_list:
    for exp in exps:
        for run in range(1,4):
            in_tuples.append((sub,exp,str(run)))

In [39]:
pool = multiprocessing.Pool(processes = 8)
pool.map(reset_art_thresh,in_tuples)
pool.terminate()
pool.join()